In [ ]:
# we need the following libraries, so let's install them
!pip install boto3
!pip install torchxrayvision
!pip install SimpleITK
!pip install radiomics
!pip install pyradiomics
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import torchxrayvision as xrv
import matplotlib.pyplot as plt
import sys
from radiomics import featureextractor
from radiomics import firstorder, glcm, glrlm, glszm, shape, shape2D, ngtdm, gldm
import warnings
import logging

warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

In [ ]:
# here we call the directories that images are saved; features are saved; targets are saved.
directory_ = '/content/sample_data/CheXpert/'
directory_processed_images = directory_ + 'processed_images'
directory_filterss = directory_ + 'filters'
directory_segments = directory_ + 'segments'
directory_cleaned_targets = directory_ + 'cleaned_targets'
directory_features = directory_ + 'features'

In [ ]:
whole_data = pd.read_csv('/content/sample_data/CheXpert/synced_data.csv')

# removing the useless column: 'Unnamed:0'
del whole_data['Unnamed: 0']
whole_data.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,-669.603949,685.781964,1.059520e+10,6.017600,866.092155,1.713989,934.117676,428.763596,10.279660,-33.372530,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-801.474548,564.486328,1.274714e+10,6.146630,954.811813,1.756523,1020.768616,468.058052,-132.958831,-191.301956,...,0.0,1.0,1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,1.0
2,-844.417249,307.704349,9.364581e+09,5.848908,690.788330,1.913630,589.552979,354.572276,-308.642302,-355.548996,...,0.0,1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0
4,-742.036096,673.796082,1.132259e+10,6.076017,1038.149048,1.627926,933.427429,467.972161,-17.909900,-36.109798,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
# checking how many NaN exist in extracted images for each images:
n_of_nans = [pd.isnull(whole_data.iloc[j,:464]).sum() for j in range(whole_data.shape[0])]

In [ ]:
# the number of images to be removed:
n_to_be_removed = (np.array(n_of_nans)>0).sum()
print("The number of images to be removed is: {0}".format(n_to_be_removed))

The number of images to be removed is: 2791


In [ ]:
whole_data_nan_free = whole_data[~pd.isnull(whole_data.iloc[:, :464]).any(axis=1)]

In [ ]:
whole_data_nan_free.to_csv('/content/sample_data/CheXpert/synced_data.csv')